<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# ================================================================
# MY-AI-Gizmo • FINAL WORKING LAUNCHER
# NO AUTO-FIXES - USES REPO AS-IS
# Complete Google Drive Persistence
# ================================================================

import os
import subprocess
import shutil
import re
import time
from pathlib import Path
from google.colab import drive

# Configuration
REPO_ZIP = "https://github.com/gitleon8301/MY-AI-Gizmo-working/archive/refs/heads/main.zip"
WORK_DIR = Path("/content/text-generation-webui")
DRIVE_ROOT = Path("/content/drive/MyDrive/MY-AI-Gizmo")

print("🔧 Setting up environment...")
# Fix matplotlib BEFORE any imports
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
os.environ['MPLBACKEND'] = 'Agg'
print("✓ Environment configured\n")

def sh(cmd):
    return subprocess.run(cmd, shell=True, capture_output=True, text=True)

def link(src, dst):
    src, dst = Path(src), Path(dst)
    if src.exists() or src.is_symlink():
        if src.is_dir() and not src.is_symlink():
            shutil.rmtree(src, ignore_errors=True)
        else:
            src.unlink(missing_ok=True)
    src.parent.mkdir(parents=True, exist_ok=True)
    src.symlink_to(dst)

print("="*60)
print("🚀 MY-AI-Gizmo Final Working Setup")
print("="*60)

# Step 1: Mount Drive
print("\n📁 Step 1/5: Mounting Google Drive...")
drive.mount("/content/drive", force_remount=False)
print("✓ Mounted")

# Step 2: Create Drive folders
print("\n💾 Step 2/5: Creating Drive structure...")
folders = ["models", "loras", "training", "characters", "presets", "prompts",
           "settings", "chat-history", "instruct-history", "outputs", "images",
           "logs", "cache", "extensions", "softprompts"]
for f in folders:
    (DRIVE_ROOT / f).mkdir(parents=True, exist_ok=True)
print(f"✓ {len(folders)} folders ready")

# Step 3: Download/setup repository
print("\n📥 Step 3/5: Setting up repository...")
if not WORK_DIR.exists():
    print("  Downloading...")
    sh("rm -f /content/repo.zip")
    sh(f"wget -q -O /content/repo.zip {REPO_ZIP}")
    sh("unzip -q /content/repo.zip -d /content")
    next(Path("/content").glob("MY-AI-Gizmo-working-*")).rename(WORK_DIR)
    print("✓ Downloaded")
else:
    print("✓ Already exists")

os.chdir(WORK_DIR)

# Step 4: Create symlinks
print("\n🔗 Step 4/5: Linking to Google Drive...")
links_map = [
    ("models", "models"), ("loras", "loras"), ("training", "training"),
    ("user_data/characters", "characters"), ("user_data/presets", "presets"),
    ("user_data/prompts", "prompts"), ("user_data/settings.yaml", "settings/settings.yaml"),
    ("user_data/settings.json", "settings/settings.json"), ("user_data/chat", "chat-history"),
    ("user_data/instruct", "instruct-history"), ("outputs", "outputs"),
    ("user_data/images", "images"), ("user_data/logs", "logs"),
    ("user_data/cache", "cache"), ("user_data/extensions", "extensions"),
    ("user_data/softprompts", "softprompts"),
]

for local, drive_folder in links_map:
    drive_path = DRIVE_ROOT / drive_folder
    if drive_path.suffix:  # It's a file
        drive_path.parent.mkdir(parents=True, exist_ok=True)
        drive_path.touch(exist_ok=True)
    else:  # It's a directory
        drive_path.mkdir(parents=True, exist_ok=True)
    link(WORK_DIR / local, drive_path)

print("✓ All data linked to Drive")

# Step 5: Install dependencies
print("\n⚙️  Step 5/5: Installing dependencies...")
print("  (First run: 5-10 min, subsequent runs: instant)\n")

sh("chmod +x start_linux.sh")
result = sh("GPU_CHOICE=A LAUNCH_AFTER_INSTALL=FALSE INSTALL_EXTENSIONS=FALSE bash start_linux.sh")

if result.returncode == 0:
    print("✓ Installation complete")
else:
    print("⚠️  Installation completed with warnings (normal)")

# ================================================================
# LAUNCH GRADIO UI
# ================================================================

print("\n" + "="*70)
print("🌐 LAUNCHING GRADIO UI")
print("="*70)
print("\n⏳ Starting server - URLs will appear below...\n")
print("="*70 + "\n")

# Kill any stuck processes
sh("pkill -9 -f 'python server.py'")
sh("pkill -9 -f 'gradio'")
time.sleep(2)

# Set up clean environment
env = os.environ.copy()
env['MPLBACKEND'] = 'Agg'
env['GRADIO_ANALYTICS_ENABLED'] = 'False'
if 'PYTHONPATH' in env:
    del env['PYTHONPATH']

# Use conda python
python_exe = WORK_DIR / "installer_files" / "env" / "bin" / "python"
if not python_exe.exists():
    python_exe = "python3"

# Launch command
cmd = f"{python_exe} server.py --share --listen"

print(f"🚀 Command: {cmd}\n")

proc = subprocess.Popen(
    cmd,
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    env=env,
    cwd=str(WORK_DIR)
)

local_url = None
public_url = None
shown = False
error_count = 0
error_lines = []

for line in proc.stdout:
    print(line, end="")

    # Track errors
    if any(x in line for x in ["Error", "Exception", "Traceback"]):
        error_lines.append(line)
        error_count += 1

    # Capture URLs
    if not public_url and (m := re.search(r"(https://[a-z0-9]+\.gradio\.live)", line)):
        public_url = m.group(1)

    if not local_url and (m := re.search(r"Running on local URL:\s+(http://[^\s]+)", line)):
        local_url = m.group(1)
    elif not local_url and (m := re.search(r"(http://(?:127\.0\.0\.1|0\.0\.0\.0):\d+)", line)):
        if ":5000" not in m.group(1):  # Skip API port
            local_url = m.group(1)

    # Show success message
    if (local_url or public_url) and not shown:
        print("\n" + "="*70)
        print("✨ SUCCESS! GRADIO UI IS RUNNING! ✨")
        print("="*70)

        if local_url:
            print(f"\n📍 LOCAL URL  : {local_url}")
            print("   ↳ Click this in Colab")

        if public_url:
            print(f"\n🌍 PUBLIC URL : {public_url}")
            print("   ↳ Share this link!")
            print("   ↳ Use on your phone!")

        print(f"\n💾 STORAGE    : {DRIVE_ROOT}")

        print("\n" + "="*70)
        print("🎯 NEXT STEPS:")
        print("="*70)
        print("1. Click the PUBLIC URL above")
        print("2. Go to 'Model' tab in the UI")
        print("3. Download a model")
        print("4. Load the model")
        print("5. Go to 'Chat' tab")
        print("6. Start chatting!")

        print("\n" + "="*70)
        print("💾 AUTO-SAVED TO DRIVE:")
        print("="*70)
        print("✓ All chat histories")
        print("✓ Character settings")
        print("✓ Downloaded models")
        print("✓ Generation presets")
        print("✓ UI preferences")
        print("✓ Images & outputs")
        print("\n💡 Re-run this script anytime - everything persists!")
        print("="*70 + "\n")

        shown = True
        error_lines = []  # Clear errors after success

# Check if process ended without URLs
if proc.poll() is not None and not shown:
    print("\n" + "="*70)
    print("⚠️  SERVER STOPPED WITHOUT LAUNCHING")
    print("="*70)
    if error_lines:
        print("\n🔍 Error detected:")
        for line in error_lines[-10:]:
            print(line, end="")
    print("\n" + "="*70)
    print("🔧 TRY THIS:")
    print("1. Restart Colab runtime (Runtime → Restart runtime)")
    print("2. Re-run this entire script")
    print("3. If error persists, see troubleshooting below")
    print("="*70)

print("\n✓ Script finished")
print(f"💾 Your data is safe in: {DRIVE_ROOT}")